In [1]:
import os
import random as rnd
import pandas as pd 
import time
import pgutils as pg
import pgmath as pm
import algorithm as al
import numpy as np

In [2]:
root_dir = r'C:\Users\eleon\PycharmProjects\SM_Exam\data\yes_small'
#root_dir = './data/yes_small'

In [8]:
#test_data=[]
with open(os.path.join(root_dir, 'test.txt')) as f:
#with open(root_dir + "\\test.txt", "r") as f:

    test_data = f.readlines()
    
test_dataset = pg.data_to_list(test_data[2:])

In [9]:
#train_data=[]
with open(os.path.join(root_dir,"train.txt"), "r") as f:
    train_data = f.readlines()
    
train_dataset = pg.data_to_list(train_data[2:])

In [23]:
train_dataset_reducted = train_dataset[:400]
print(np.max(np.array(train_dataset_reducted)))
max(train_dataset_reducted)

[2199]


[2199]

In [3]:
song_hash = pd.read_csv(os.path.join(root_dir,"song_hash.txt"), sep="\t", header=None)


In [14]:
#toy_test=[[0,4,5,3,6],[4,3,2,6],[0,2,1,0,5]]
# evaluate test performance using the average log-likelihood as metric
# it is defined as log(Pr(d_test))/n_test)
# n_test number of transition in the test set
#songs = 7 #len(song_hash)
songs=2200
n_test = np.sum(pg.transition_count(songs,test_dataset[:100]))
print(n_test)
#evaluation = pg.log_like(test_dataset[:100],prob_matrix) /n_test
#print(evaluation)
#result = sum( sum(dimension.D[s, i.index()] ) for i in range(len(test_dataset)))  

148.0


In [5]:
x = np.genfromtxt(r'C:\Users\eleon\Desktop\SM_Exam\src\latent_representation2200.csv', delimiter=' ')
x.shape


(2200, 10)

In [6]:
#dummy_landmarks = [[i for i in range(songs)] for j in range(songs)]
d2 = pm.Distances.delta(x)
d2.shape

(2200, 2200)

In [19]:
dummy_landmarks = [[i for i in range(2200)] for j in range(2200)]
prob_matrix = np.exp(-d2) / pm.Distances.zeta(d2, x, dummy_landmarks)[:, np.newaxis]
prob_matrix.shape

(2200, 2200)

In [20]:
cum_sum = np.sum(prob_matrix, axis=1)
prob_matrix = prob_matrix / cum_sum[:, np.newaxis]

In [21]:
from numpy import savetxt

savetxt('prob_matrix_2200.csv', prob_matrix, delimiter=' ')


In [28]:
pg.playlist_generator(10,70, prob_matrix,song_hash)

Song in the playlist:  ["Blowin' Money Fast (B.M.F.) (w\\/ Styles P)", 'Run To The Hills', 'Dangerous (w\\/ Akon)', 'Keep Your Hands To Yourself', 'Best Of You', 'Everything', 'Tijuana Dance', 'Live And Let Die', 'OMG (w\\/ Will.I.Am)', 'Straight Up', 'Chasing Pirates']
End state after  10  days:  Chasing Pirates
Probability of the possible sequence of states:  2.4099750917556775e-34
